In [2]:
import pandas as pd
from scipy.io import arff

# Đọc file ARFF
data, meta = arff.loadarff('elecNormNew.arff')

# Chuyển đổi thành DataFrame
df = pd.DataFrame(data)

# Hiển thị DataFrame
df.head()

,date,day,period,nswprice,nswdemand,vicprice,vicdemand,transfer,class
0,0.0,b'2',0.000000,0.056443,0.439155,0.003467,0.422915,0.414912,b'UP'
1,0.0,b'2',0.021277,0.051699,0.415055,0.003467,0.422915,0.414912,b'UP'
2,0.0,b'2',0.042553,0.051489,0.385004,0.003467,0.422915,0.414912,b'UP'
3,0.0,b'2',0.063830,0.045485,0.314639,0.003467,0.422915,0.414912,b'UP'
4,0.0,b'2',0.085106,0.042482,0.251116,0.003467,0.422915,0.414912,b'DOWN'


In [4]:
from sklearn.preprocessing import LabelEncoder, StandardScaler

# Chuyển đổi cột 'day' và 'class' từ kiểu object sang kiểu số
label_encoder_day = LabelEncoder()
df['day'] = label_encoder_day.fit_transform(df['day'])

label_encoder_class = LabelEncoder()
df['class'] = label_encoder_class.fit_transform(df['class'])

# Kiểm tra và xử lý các giá trị thiếu (nếu có)
df = df.fillna(df.mean())

# Hiển thị DataFrame đã được tiền xử lý
df.head()

,date,day,period,nswprice,nswdemand,vicprice,vicdemand,transfer,class
0,0.0,1,0.000000,0.056443,0.439155,0.003467,0.422915,0.414912,1
1,0.0,1,0.021277,0.051699,0.415055,0.003467,0.422915,0.414912,1
2,0.0,1,0.042553,0.051489,0.385004,0.003467,0.422915,0.414912,1
3,0.0,1,0.063830,0.045485,0.314639,0.003467,0.422915,0.414912,1
4,0.0,1,0.085106,0.042482,0.251116,0.003467,0.422915,0.414912,0


In [ ]:
from sklearn.ensemble import RandomForestClassifier
from collections import deque
import numpy as np
import pandas as pd
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score
import matplotlib.pyplot as plt

In [ ]:
def create_datastream(data, batch_size):
    datastream = []
    for i in range(0, len(data), batch_size):
        batch = data.iloc[i:i+batch_size]
        datastream.append(batch)
    return datastream

In [ ]:
def train_random_forest_classifier(X, y, n_estimators=50):
    """
    Huấn luyện bộ phân loại Random forest trên dữ liệu X, y.
    n_estimators: Số lượng cây trong Random forest.
    """
    rf = RandomForestClassifier(n_estimators=n_estimators)
    rf.fit(X, y)
    return rf

In [ ]:
def adaptive_ensemble_size(C, sample, unique_label, anpha, min_num=3):
    """
    Cơ chế adaptive_esemble 
    C : Bộ phân loại yếu
    sample
    unique_label : Tập hợp nhãn trong luồng dữ liệu
    anpha : hyper parameter
    min_num : số lượng C tối thiểu 
    """
    probability_list = []
    # Tính toán xác suất dự đoán cho mỗi bộ phân loại trong C và lưu trữ kết quả
    sample_proba = [clf.predict_proba(sample)[0] for clf in C]
    
    for label in unique_label:
        li = []
        latest_proba = None
        
        for proba in sample_proba:
            # Tạo từ điển để ánh xạ nhãn với xác suất tương ứng
            probability_dict = {lbl: prob for lbl, prob in zip(C[0].classes_, proba)}
            # Lấy xác suất hiện tại cho nhãn đang xét
            current_proba = probability_dict.get(label, 0)
            
            if len(li) < min_num:
                # Nếu số lượng xác suất trong li nhỏ hơn min_num, thêm current_proba vào li
                li.append(current_proba)
            else:
                if latest_proba is None:
                    # Nếu latest_proba chưa được gán giá trị, gán giá trị của nó bằng xác suất mới nhất
                    latest_proba = probability_dict.get(label, 0)
                
                # So sánh xác suất hiện tại với xác suất mới nhất
                if abs(current_proba - latest_proba) < anpha:
                    # Nếu chênh lệch nhỏ hơn anpha, thêm current_proba vào li
                    li.append(current_proba)
                else:
                    # Nếu chênh lệch lớn hơn anpha, dừng vòng lặp
                    break
        
        # Thêm danh sách xác suất cho nhãn hiện tại vào probability_list
        probability_list.append(li)
    
    return probability_list

In [ ]:
def linear_regression(x, y):
    """Tính hồi quy tuyến tính"""
    A = np.vstack([x, np.ones(len(x))]).T
    m, c = np.linalg.lstsq(A, y, rcond=None)[0]
    return m, c

def tendency_prediction(probability_list, Y, epsilon=0.01):
    """
    Thực hiện cơ chế dự đoán xu hướng
    Input:
    - probability_list: Danh sách xác suất cho mỗi mẫu
    - Y: Danh sách các lớp trong luồng dữ liệu
    - epsilon: Hệ số trọng số (mặc định là 0.01)
    Output:
    - Ps: Lớp dự đoán cho mẫu
    """
    predicted_probabilities = []
    
    for i, li in enumerate(probability_list):
        x = np.arange(1, len(li) + 1)
        y = np.array(li)
        slope, intercept = linear_regression(x, y)
        next_value = slope * (len(li) + 2) + intercept
        li = [next_value] + li  # Thêm giá trị dự đoán tiếp theo vào đầu danh sách
        weights = 1 - np.arange(1, len(li) + 1) * epsilon
        weighted_prob = np.dot(li, weights) / len(li)
        predicted_probabilities.append(weighted_prob)
    
    Ps = Y[np.argmax(predicted_probabilities)]
    
    return Ps


In [ ]:
def process_data_stream(S, m, k, unique_labels):
    C = deque(maxlen=m)
    true_labels = []
    predicted_labels = []
    pre = []
    block_accuracies = []  # Danh sách để lưu trữ độ chính xác của từng block

    for i in range(len(S) - 1):
        Bi = S[i]
        block_predictions = []
        X = Bi.iloc[:, :-1]
        y = Bi.iloc[:, -1]
        Ci = train_random_forest_classifier(X, y)
        C.append(Ci)
        if len(C) < k:
            continue
        Bi_1 = S[i + 1]
        for index, row in Bi_1.iterrows():
            sample = pd.DataFrame([row[:-1]], columns=Bi.columns[:-1])
            anpha = (1500 / len(Bi_1)) * 0.2
            selected_classifiers = adaptive_ensemble_size(C, sample, unique_labels, anpha)
            pre_sample = tendency_prediction(selected_classifiers, unique_labels)
            block_predictions.append(pre_sample)
            true_labels.append(row.iloc[-1])
            predicted_labels.append(pre_sample)
        pre.append(block_predictions)

        # Tính toán độ chính xác cho block hiện tại
        block_accuracy = accuracy_score(true_labels[-len(Bi_1):], predicted_labels[-len(Bi_1):])
        block_accuracies.append(block_accuracy)
    
    precision = precision_score(true_labels, predicted_labels, average='weighted')
    recall = recall_score(true_labels, predicted_labels, average='weighted')
    f1 = f1_score(true_labels, predicted_labels, average='weighted')
    accuracy = accuracy_score(true_labels, predicted_labels)
    
    print(f"Precision: {precision}")
    print(f"Recall: {recall}")
    print(f"F1-Score: {f1}")
    print(f"Accuracy: {accuracy}")

    # Trực quan hóa độ chính xác của từng block
    plt.figure(figsize=(10, 6))
    plt.plot(block_accuracies, marker='o', linestyle='-', color='b')
    plt.title('Accuracy of Each Block')
    plt.xlabel('Block Index')
    plt.ylabel('Accuracy')
    plt.grid(True)
    plt.show()

# Main execution
batch_size = 500
S = create_datastream(df, batch_size)
m = 15
k = 3
unique_labels = list(set(df.iloc[:, -1]))
process_data_stream(S, m, k, unique_labels)